In [3]:
import re

In [4]:
set_data_structure = ''

In [5]:
def line_by_line(input_file):
    f = open(input_file, "r")
    arr = f.read().split('\n')

    # Remove blanks
    arr = list(filter(None, arr))

    # trim each element
    arr = [x.strip() for x in arr]

    return arr


In [6]:
line_by_line("input.py")

['tack  = []',
 'def top():',
 'global stack',
 'if len(stack) > 1:',
 'return stack[-1]',
 'return -1',
 'def pop():',
 'global stack',
 'popped = stack[-1]',
 'stack = stack[:len(stack)-1]',
 'return popped',
 'def push(a: int):',
 'global stack',
 'stack.append(a)',
 'def multi_pop(num: int):',
 'global stack',
 'for i in range(min(len(stack), num)):',
 'pop()',
 "if __name__ == '__main__':",
 'print("Hello World")',
 'push(1)',
 'push(2)',
 'pop()',
 'push(24)',
 'pop()',
 'push(345)',
 'push(2)',
 'pop()',
 'multi_pop(100)']

In [7]:
# Pattern matching for stack
stack_str = "(def top\(\):|def pop\(\):|def push\(a: int\):)"

In [8]:
data_structure_dict = {"stack": {"name": 'stack', 'regex_str': stack_str}}
print(data_structure_dict)

{'stack': {'name': 'stack', 'regex_str': '(def top\\(\\):|def pop\\(\\):|def push\\(a: int\\):)'}}


In [9]:
for ds in data_structure_dict:
    a = data_structure_dict[ds]
    re_str = a['regex_str']
    
    x = re.search(re_str, 'def pop():')
    if x:
        print('Data Structure Match')
    else:
        print('No Match')

Data Structure Match


In [10]:
def identity_datastructure(input_str):
    for ds in data_structure_dict:
        a = data_structure_dict[ds]
        re_str = a['regex_str']
        
        x = re.search(re_str, input_str)
        if x:
            return {'success': True, 'ds': a}
        else:
            pass
    return {'success': False}


In [11]:
for line in line_by_line('input.py'):
    result = identity_datastructure(line)
    print(result)
    if result['success']:
        print(result['ds'])
        break


{'success': False}
{'success': True, 'ds': {'name': 'stack', 'regex_str': '(def top\\(\\):|def pop\\(\\):|def push\\(a: int\\):)'}}
{'name': 'stack', 'regex_str': '(def top\\(\\):|def pop\\(\\):|def push\\(a: int\\):)'}


In [42]:
def lines_after_main(arr):
    for i in range(len(arr)):
        if arr[i] == "if __name__ == '__main__':":
            driver = arr[i+1:]
    return driver

In [32]:
driver_code = lines_after_main(line_by_line('input.py'))

['tack  = []', 'def top():', 'global stack', 'if len(stack) > 1:', 'return stack[-1]', 'return -1', 'def pop():', 'global stack', 'popped = stack[-1]', 'stack = stack[:len(stack)-1]', 'return popped', 'def push(a: int):', 'global stack', 'stack.append(a)', 'def multi_pop(num: int):', 'global stack', 'for i in range(min(len(stack), num)):', 'pop()', "if __name__ == '__main__':", 'print("Hello World")', 'push(1)', 'push(2)', 'pop()', 'push(24)', 'pop()', 'multi_pop(3)', 'push(345)', 'push(2)', 'pop()', 'push(2)', 'push(2)', 'push(2)', 'push(2)', 'multi_pop(100)']


In [33]:
driver_code

['print("Hello World")',
 'push(1)',
 'push(2)',
 'pop()',
 'push(24)',
 'pop()',
 'multi_pop(3)',
 'push(345)',
 'push(2)',
 'pop()',
 'push(2)',
 'push(2)',
 'push(2)',
 'push(2)',
 'multi_pop(100)']

In [40]:
count = 0
num_vars = 0
for line in driver_code:
    if re.search('^push.*', line):
        num_vars += 1
    elif re.search('^pop.*', line):
        num_vars -= 1
    elif re.search('^multi.*', line):
        num_arr = [s for s in line if s.isdigit()]
        num_mp = int(''.join(num_arr))
        pop_exec= min(num_mp, num_vars)
        count += pop_exec
        num_vars -= pop_exec
    count += 1


In [25]:
num_vars

0

In [26]:
count

12

## Aggregate Analysis

In [43]:
driver_code = lines_after_main(line_by_line('input.py'))

def aggregate_analysis(driver_code: list):
    # variable to keep track of number of lines
    count = 0
    # varibale to keep track of the number of variables
    num_vars = 0
    # variable to keep track of number of excecutions
    num_execs = 0

    # parse every line
    for line in driver_code:
        if re.search('^push.*', line):
            num_vars += 1
            num_execs += 1
        elif re.search('^pop.*', line):
            num_vars -= 1
            num_execs += 1
        elif re.search('^multi.*', line):
            num_arr = [s for s in line if s.isdigit()]
            num_mp = int(''.join(num_arr))
            pop_exec= min(num_mp, num_vars)
            num_execs = num_execs + pop_exec
            count += pop_exec
            num_vars -= pop_exec
        count += 1

    print(f"Total number of executions: {num_execs}")
    print(f"Number of commands: {count}")
    print(f"Total cost: {num_execs / count}")

In [44]:
aggregate_analysis(driver_code)

Total number of executions: 18
Number of commands: 21
Total cost: 0.8571428571428571


## Accounting Methods

#### Ammortized Costs
- Cost assigned to Push: 2
- Cost assigned to Pop: 0
- Cost assigned to Multipop: 0

In [52]:
driver_code = lines_after_main(line_by_line('input.py'))

def accounting(driver_code: list):
    # variable to keep track of number of lines
    count = 0
    # varibale to keep track of the number of variables
    num_vars = 0
    # variable to keep track of number of excecutions
    num_execs = 0
    # variable to keep track of balance
    bal = 0

    # parse every line
    for line in driver_code:
        if re.search('^push.*', line):
            num_vars += 1
            # Ammortized cost of Pop: 2 and real cost: 1
            bal = bal + 2 - 1
        elif re.search('^pop.*', line):
            num_vars -= 1
            # Ammortized cost of Pop: 0 and real cost: 1
            bal = bal -1
        elif re.search('^multi.*', line):
            num_arr = [s for s in line if s.isdigit()]
            num_mp = int(''.join(num_arr))
            pop_exec= min(num_mp, num_vars)
            num_vars -= pop_exec
            # Ammortized cost of multipop: 0 and real cost: pop_exec
            bal = bal - pop_exec
        count += 1
        print(f"Balance after {count} is {bal}")

In [53]:
accounting(driver_code)

Balance after 1 is 0
Balance after 2 is 1
Balance after 3 is 2
Balance after 4 is 1
Balance after 5 is 2
Balance after 6 is 1
Balance after 7 is 0
Balance after 8 is 1
Balance after 9 is 2
Balance after 10 is 1
Balance after 11 is 2
Balance after 12 is 3
Balance after 13 is 4
Balance after 14 is 5
Balance after 15 is 0


## Potential Method

In [59]:
driver_code = lines_after_main(line_by_line('input.py'))

def accounting(driver_code: list):
    # variable to keep track of number of lines
    count = 0
    # varibale to keep track of the number of variables
    num_vars = 0
    # variable to keep track of number of excecutions
    num_execs = 0
    # variable to keep track of balance
    ammortized_cost = 0
    # Total cost
    total = 0

    # parse every line
    for line in driver_code:
        if re.search('^push.*', line):
            # Ammortized cost of Pop: 2 and real cost: 1
            ammortized_cost = ammortized_cost +  1 + (num_vars + 1 - num_vars)
            num_vars += 1
            total += 1
        elif re.search('^pop.*', line):
            # Ammortized cost of Pop: 0 and real cost: 1
            ammortized_cost = ammortized_cost - 1 + (num_vars - 1 - num_vars)
            num_vars -= 1
            total += 1
        elif re.search('^multi.*', line):
            num_arr = [s for s in line if s.isdigit()]
            num_mp = int(''.join(num_arr))
            pop_exec= min(num_mp, num_vars)
            # Ammortized cost of multipop: 0 and real cost: pop_exec
            ammortized_cost = ammortized_cost + pop_exec - (pop_exec)
            num_vars -= pop_exec
            total += pop_exec
        count += 1
        print(f"ammortized_costance after {count} is {ammortized_cost}")
    # print(f"Total cost is {total}")

In [60]:
accounting(driver_code)

ammortized_costance after 1 is 0
ammortized_costance after 2 is 2
ammortized_costance after 3 is 4
ammortized_costance after 4 is 2
ammortized_costance after 5 is 4
ammortized_costance after 6 is 2
ammortized_costance after 7 is 2
ammortized_costance after 8 is 4
ammortized_costance after 9 is 6
ammortized_costance after 10 is 4
ammortized_costance after 11 is 6
ammortized_costance after 12 is 8
ammortized_costance after 13 is 10
ammortized_costance after 14 is 12
ammortized_costance after 15 is 12
